In [1]:
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from numpy import load
from torch.utils.data import Dataset
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
from timeit import default_timer as timer
import torch
from torch.autograd import Variable
import torch.nn as nn
import numpy as np
from datetime import datetime

In [2]:


def batch_matmul(seq, weight, nonlinearity=''):
    s = None
    for i in range(seq.size(0)):
        _s = torch.mm(seq[i], weight)
        if(nonlinearity=='tanh'):
            _s = torch.tanh(_s)
        _s = _s.unsqueeze(0)
        if(s is None):
            s = _s
        else:
            s = torch.cat((s,_s),0)
    return s.squeeze()

class RNNModel(nn.Module):
    """Container module with an encoder, a recurrent module, and a decoder."""

    def __init__(self, rnn_type, ntoken, ninp, nhid, nlayers,proj_size, dropout=0.5, tie_weights=False, attention=True,
                 attention_width=3, cuda=True):
        super(RNNModel, self).__init__()
        self.drop = nn.Dropout(dropout)
#         self.encoder = nn.Embedding(ntoken, ninp)
        self.encoder = nn.Linear(ntoken, ninp)
        if rnn_type in ['LSTM', 'GRU']:
            self.rnn = getattr(nn, rnn_type)(ninp, nhid, nlayers, dropout=dropout)
        else:
            try:
                nonlinearity = {'RNN_TANH': 'tanh', 'RNN_RELU': 'relu'}[rnn_type]
            except KeyError:
                raise ValueError("""An invalid option for `--model` was supplied,
                                 options are ['LSTM', 'GRU', 'RNN_TANH' or 'RNN_RELU']""")
            self.rnn = nn.RNN(ninp, nhid, nlayers, nonlinearity=nonlinearity, dropout=dropout)
        if attention:
            self.decoder = nn.Linear(nhid, ntoken)
        else:
            self.decoder = nn.Linear(nhid, ntoken)
        if tie_weights:
            if nhid != ninp:
                raise ValueError('When using the tied flag, nhid must be equal to emsize')
            self.decoder.weight = self.encoder.weight

        self.softmax = nn.Softmax()
        if attention:
            self.AttentionLayer = AttentionLayer(cuda,nhid)
        self.init_weights()

        self.rnn_type = rnn_type
        self.nhid = nhid
        self.nlayers = nlayers
        self.attention = attention
        self.attention_width = attention_width
        self.generator = nn.Linear(nhid, proj_size)
    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.fill_(0)
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, input):
        #print("input size:",input.size())
#         emb = self.drop(self.encoder(input))
        emb=self.encoder(input)
        #print("emb size:",emb.size())
        output, _ = self.rnn(emb)
        #print("rnn output",output.size())
        if self.attention:
            output = self.AttentionLayer.forward(output, self.attention_width)
        return self.generator(output)
        #
#         output = self.drop(output)
#         decoded = self.decoder(output.view(output.size(0) * output.size(1), output.size(2)))
#         return decoded.view(output.size(0), output.size(1), decoded.size(1)), hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters()).data
        if self.rnn_type == 'LSTM':
            return (Variable(weight.new(self.nlayers, bsz, self.nhid).zero_()),
                    Variable(weight.new(self.nlayers, bsz, self.nhid).zero_()))
        else:
            return Variable(weight.new(self.nlayers, bsz, self.nhid).zero_())


class AttentionLayer(nn.Module):
    """Implements an Attention Layer"""

    def __init__(self, cuda, nhid):
        super(AttentionLayer, self).__init__()
        self.nhid = nhid
        self.weight_W = nn.Parameter(torch.Tensor(nhid,nhid))
        self.weight_proj = nn.Parameter(torch.Tensor(nhid, 1))
        self.softmax = nn.Softmax()
        self.weight_W.data.uniform_(-0.1, 0.1)
        self.weight_proj.data.uniform_(-0.1,0.1)
        self.cuda = cuda

    def forward(self, inputs, attention_width=3):
        results = None
        for i in range(inputs.size(0)):
            if(i<attention_width):
                output = inputs[i]
                output = output.unsqueeze(0)

            else:
                lb = i - attention_width
                if(lb<0):
                    lb = 0
                selector = torch.from_numpy(np.array(np.arange(lb, i)))
                if self.cuda:
                    selector = Variable(selector).cuda()
                else:
                    selector = Variable(selector)
                vec = torch.index_select(inputs, 0, selector)
                u = batch_matmul(vec, self.weight_W, nonlinearity='tanh')
                a = batch_matmul(u, self.weight_proj)
                a = self.softmax(a)
                output = None
                for i in range(vec.size(0)):
                    h_i = vec[i]
                    a_i = a[i].unsqueeze(1).expand_as(h_i)
                    h_i = a_i * h_i
                    h_i = h_i.unsqueeze(0)
                    if(output is None):
                        output = h_i
                    else:
                        output = torch.cat((output,h_i),0)
                output = torch.sum(output,0)
                output = output.unsqueeze(0)

            if(results is None):
                results = output

            else:
                results = torch.cat((results,output),0)

        return results

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



def batchify(data, bsz,bptt):
    # 데이터셋을 bsz 파트들로 나눕니다.
    if(data.size(0)%(bsz*bptt)!=0):
        
#         print(data.size(0),bsz,bptt)
        
        data=data.view(-1,bptt,data.size(1)).transpose(0,1).contiguous()
        return data.to(device)
    nbatch = data.size(0) // bsz
    # 깔끔하게 나누어 떨어지지 않는 추가적인 부분(나머지들) 은 잘라냅니다.
    data = data.narrow(0, 0, nbatch * bsz)
    # 데이터에 대하여 bsz 배치들로 동등하게 나눕니다.
    data = data.view(bsz, -1,data.size(1)).transpose(0,1).contiguous()
    return data.to(device)


In [4]:


bptt = 39
def get_batch(source, i,bs):
    seq_len = min(bptt*bs, len(source)  - i)
    data = source[i:i+seq_len]
#     if(seq_len!=bptt*bs):
#         print(seq_len)
    target = source[i:i+seq_len].reshape(-1)
    return data, target


In [5]:
#from torch.utils.data import DataLoader

def train_epoch(model, optimizer):
    model.train()
    losses = 0
    total=0
    correct=0
    correct1=0
    tot1=0
    correct2=0
    tot2=0
    correct0=0
    tot0=0
    conf00=0
    conf01=0
    conf02=0
    conf10=0
    conf11=0
    conf12=0
    conf20=0
    conf21=0
    conf22=0
    
    
#     for batch, i in enumerate(range(0, Xtrain_data.size(0) - 1, BATCH_SIZE*bptt)):
    i=0
    while(i<Xtest_data.size(0)):
        data, org_targets = get_batch(Xtrain_data, i,BATCH_SIZE)
        if (data.isnan().any() or data.isinf().any()):
#            print(data)
            continue
#         _,targets = get_batch(Ytrain_data,i)
        targets,_ = get_batch(Ytrain_data,i,BATCH_SIZE)
#         src = src.to(DEVICE)
#         tgt = tgt.to(DEVICE)
        targets=torch.unsqueeze(targets,1)
        src=batchify(data,BATCH_SIZE,bptt)
        tgt=batchify(targets,BATCH_SIZE,bptt)
        src=src[:-1]
        tgt=tgt[1:]
        logits = model(src)

        optimizer.zero_grad()

        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()
        
        _,predicted=torch.max(logits,-1)
        correct += (tgt.squeeze() == predicted).sum().item()
        total+=len(predicted)*BATCH_SIZE
        tot0+=(0== tgt.squeeze()).sum().item()
        tot1+=(1== tgt.squeeze()).sum().item()
        tot2+=(2== tgt.squeeze()).sum().item()
        correct0+=((0== predicted) &(0==tgt.squeeze())).sum().item()
        correct1+=((1== predicted)&(1==tgt.squeeze())).sum().item()
        correct2+=((2== predicted)&(2==tgt.squeeze())).sum().item()
        
        conf00+=((0== predicted) &(0==tgt.squeeze())).sum().item()
        conf01+=((0== predicted)&(1==tgt.squeeze())).sum().item()
        conf02+=((0== predicted)&(2==tgt.squeeze())).sum().item()
        conf10+=((1== predicted) &(0==tgt.squeeze())).sum().item()
        conf11+=((1== predicted)&(1==tgt.squeeze())).sum().item()
        conf12+=((1== predicted)&(2==tgt.squeeze())).sum().item()
        conf20+=((2== predicted) &(0==tgt.squeeze())).sum().item()
        conf21+=((2== predicted)&(1==tgt.squeeze())).sum().item()
        conf22+=((2== predicted)&(2==tgt.squeeze())).sum().item()

        
        
        
        i+=targets.size()[0]

    tp0=conf00
    fp0=conf01+conf02
    fn0=conf10+conf20
    if(tp0+fp0==0):
        prec0=0
    else:
        prec0=tp0/(tp0+fp0)
    if(tp0+fn0==0):
        reca0=0
    else:
        reca0=tp0/(tp0+fn0)
    
    tp1=conf11
    fp1=conf10+conf12
    fn1=conf01+conf21
    
    if(tp1+fp1==0):
        prec1=0
    else:
        prec1=tp1/(tp1+fp1)
    if(tp1+fn1==0):
        reca1=0
    else:
        reca1=tp1/(tp1+fn1)
    
    tp2=conf22
    fp2=conf20+conf21
    fn2=conf02+conf12
    
    if(tp2+fp2==0):
        prec2=0
    else:
        prec2=tp2/(tp2+fp2)
    if(tp2+fn2==0):
        reca2=0
    else:
        reca2=tp2/(tp2+fn2)
        
    prec=(prec0+prec1+prec2)/3
    reca=(reca0+reca1+reca2)/3
    f1sc=2*(prec*reca)/(prec+reca)
    print(total)
    print(correct)
    print("Acc:",correct/total)            
    print("Prec",prec)
    print("Recall",reca)
    print("F1",f1sc) 
    return losses / Xtrain_data.size(0),[conf00,conf01,conf02,conf10,conf11,conf12,conf20,conf21,conf22]

def evaluate(model):
    model.eval()
    losses = 0
    total=0
    correct=0
    correct1=0
    tot1=0
    correct2=0
    tot2=0
    correct0=0
    tot0=0
    conf00=0
    conf01=0
    conf02=0
    conf10=0
    conf11=0
    conf12=0
    conf20=0
    conf21=0
    conf22=0
    with torch.no_grad():
        
        i=0
        while(i<Xtest_data.size(0)):
            data, org_targets = get_batch(Xtest_data, i,BATCH_SIZE)
            if (data.isnan().any() or data.isinf().any()):
                print(data)
                continue
            targets,_ = get_batch(Ytest_data,i,BATCH_SIZE)
            targets=torch.unsqueeze(targets,1)
            src=batchify(data,BATCH_SIZE,bptt)
            tgt=batchify(targets,BATCH_SIZE,bptt)
#             print(src.shape)
            if(src.shape[1]!=BATCH_SIZE):
                
                break
#             print(tgt.shape)
            src=src[:-1]
            tgt=tgt[1:]


            logits = model(src)
#            print(logits.reshape(-1, logits.shape[-1]).shape,'AA')
#            print(tgt.reshape(-1).shape,'BB')
            loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt.reshape(-1))
            if(loss.isnan()):
                print(src,tgt_input)
                break
            losses += loss.item()
            _,predicted=torch.max(logits,-1)

            
            correct += (tgt.squeeze() == predicted).sum().item()
            total+=len(predicted)*BATCH_SIZE
            tot0+=(0== tgt.squeeze()).sum().item()
            tot1+=(1== tgt.squeeze()).sum().item()
            tot2+=(2== tgt.squeeze()).sum().item()
            correct0+=((0== predicted) &(0==tgt.squeeze())).sum().item()
            correct1+=((1== predicted)&(1==tgt.squeeze())).sum().item()
            correct2+=((2== predicted)&(2==tgt.squeeze())).sum().item()
            
            conf00+=((0== predicted) &(0==tgt.squeeze())).sum().item()
            conf01+=((0== predicted)&(1==tgt.squeeze())).sum().item()
            conf02+=((0== predicted)&(2==tgt.squeeze())).sum().item()
            conf10+=((1== predicted) &(0==tgt.squeeze())).sum().item()
            conf11+=((1== predicted)&(1==tgt.squeeze())).sum().item()
            conf12+=((1== predicted)&(2==tgt.squeeze())).sum().item()
            conf20+=((2== predicted) &(0==tgt.squeeze())).sum().item()
            conf21+=((2== predicted)&(1==tgt.squeeze())).sum().item()
            conf22+=((2== predicted)&(2==tgt.squeeze())).sum().item()

            
            i+=targets.size()[0]
            
#     print(total,tot0,tot1,tot2)
#     print(correct)
    tp0=conf00
    fp0=conf01+conf02
    fn0=conf10+conf20
    if(tp0+fp0==0):
        prec0=0
    else:
        prec0=tp0/(tp0+fp0)
    if(tp0+fn0==0):
        reca0=0
    else:
        reca0=tp0/(tp0+fn0)
    
    tp1=conf11
    fp1=conf10+conf12
    fn1=conf01+conf21
    
    if(tp1+fp1==0):
        prec1=0
    else:
        prec1=tp1/(tp1+fp1)
    if(tp1+fn1==0):
        reca1=0
    else:
        reca1=tp1/(tp1+fn1)
    
    tp2=conf22
    fp2=conf20+conf21
    fn2=conf02+conf12
    
    if(tp2+fp2==0):
        prec2=0
    else:
        prec2=tp2/(tp2+fp2)
    if(tp2+fn2==0):
        reca2=0
    else:
        reca2=tp2/(tp2+fn2)
    
    prec=(prec0+prec1+prec2)/3
    reca=(reca0+reca1+reca2)/3
    if(prec+reca==0):
        f1sc=0
    else:
        f1sc=2*(prec*reca)/(prec+reca)
    print(total)
    print(correct)
    print("Acc:",correct/total)            
    print("Prec",prec)
    print("Recall",reca)
    print("F1",f1sc)
    return losses / Xtest_data.size(0),correct/total,prec,reca,f1sc,[conf00,conf01,conf02,conf10,conf11,conf12,conf20,conf21,conf22]

In [ ]:

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

mbrnlist1=[(5,194),(2,155),(12,100),(17,29),(42,1),(44,1),(50,92),(2,83),(4,10118),(8,298)]
mbrnlist2=[(5,194),(12,100),(2,155),(17,29),(42,1),(44,1),(2,83),(4,10118),(4,9997),(50,91)]

mbrnlist=mbrnlist1+mbrnlist2
mbrnlist=set(mbrnlist)
mbrnlist=list(mbrnlist)

# load array
MBR_NO,BRN_NO=mbrnlist[11]
# print the array

for MBR_NO,BRN_NO in mbrnlist:

    Data_train = load('Data0930_'+str(MBR_NO)+'_'+str(BRN_NO)+'norm_train.npy',allow_pickle=True)
    Data_test =  load('Data0930_'+str(MBR_NO)+'_'+str(BRN_NO)+'norm_test.npy',allow_pickle=True)


    Xdata=[]
    Ydata=[]
    Xtrain_data=[]
    Ytrain_data=[]
    Xtest_data=[]
    Ytest_data=[]



    for idx in range(len(Data_train)//39):
        if(np.isinf(Data_train[39*idx:39*(idx+1)][:,:].tolist()).any()):
            print(np.isinf(Data_train[39*idx:39*(idx+1)][:,:].tolist()).any())
            continue
        Xtrain_data.append(Data_train[39*idx:39*(idx+1)][:,:].tolist())
        Ytrain_data.append(Data_train[39*idx:39*(idx+1)][:,-1].tolist())
    for idx in range(len(Data_test)//39):
        if(np.isinf(Data_test[39*idx:39*(idx+1)][:,:].tolist()).any()):
            print(np.isinf(Data_test[39*idx:39*(idx+1)][:,:].tolist()).any())
            continue
        Xtest_data.append(Data_test[39*idx:39*(idx+1)][:,:].tolist())
        Ytest_data.append(Data_test[39*idx:39*(idx+1)][:,-1].tolist())

    Xtrain_data=np.vstack(Xtrain_data)
    Ytrain_data=np.vstack(Ytrain_data)

    Xtrain_data=torch.FloatTensor(Xtrain_data)
    Ytrain_data=torch.IntTensor(Ytrain_data)
    Ytrain_data=Ytrain_data.view(-1)


    Xtest_data=np.vstack(Xtest_data)
    Ytest_data=np.vstack(Ytest_data)
    Xtest_data=torch.FloatTensor(Xtest_data)
    Ytest_data=torch.IntTensor(Ytest_data)
    Ytest_data=Ytest_data.view(-1)

    Ytrain_data=2*(Ytrain_data>0).long()+(Ytrain_data==0).long()
    Ytest_data=2*(Ytest_data>0).long()+(Ytest_data==0).long()
    Ytrain_data=Ytrain_data.T
    Ytest_data=Ytest_data.T


    INPUT_SIZE=Xtrain_data.shape[1]
    EMB_SIZE=64
    HIDDEN_SIZE=64
    NUM_LAYERS=6
    PROJ_SIZE=3
    BATCH_SIZE=16
    torch.manual_seed(0)



    #lstm=Seq2SeqLSTM(input_size=INPUT_SIZE,hidden_size=HIDDEN_SIZE,num_layers=NUM_LAYERS,emb_size=EMB_SIZE,proj_size=PROJ_SIZE)

    #lstm = RNNmodel(rnn_type='LSTM',ntoken=INPUT_SIZE,ninp=INPUT_SIZE,nhid=HIDDEN_SIZE,nlayers=NUM_LAYERS,emb_size=EMB_SIZE,proj_size=PROJ_SIZE)
    lstm = RNNModel(rnn_type='LSTM',ntoken=INPUT_SIZE,ninp=EMB_SIZE,nhid=HIDDEN_SIZE,nlayers=NUM_LAYERS,proj_size=PROJ_SIZE,
                    attention_width=38)
    # for p in lstm.parameters():
    #     if p.dim() > 1:
    #         nn.init.xavier_uniform_(p)

    lstm = lstm.to(DEVICE)

    loss_fn = torch.nn.CrossEntropyLoss()

    optimizer = torch.optim.Adam(lstm.parameters(), lr=1e-3, betas=(0.9, 0.98), eps=1e-9)


    NUM_EPOCHS = 1000
    best_val_loss=100000000
    for epoch in range(1, NUM_EPOCHS+1):
        start_time = timer()
        train_loss,_ = train_epoch(lstm, optimizer)
        end_time = timer()
        val_loss,acc,prec,reca,f1sc,confusion = evaluate(lstm)
        print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_confusion=confusion
            best_acc=acc
            best_prec=prec
            best_reca=reca
            best_f1sc=f1sc
            best_model = lstm

    now = datetime.now()
    now.strftime("%m/%d/%Y, %H:%M:%S")

    date_time = now.strftime("%m_%d_%Y")
    PATH='best_model_seq_attLSTM_'+date_time+str(MBR_NO)+'_'+str(BRN_NO)
    torch.save(best_model.state_dict(), PATH)
    file_name='result_attLSTM_norm'+date_time+'.txt'
    text_to_append=PATH+'\t'+"Acc:"+str(best_acc)+'\t'+"prec:"+str(best_prec)+'\t'+"recall:"+str(best_reca)+'\t'+"f1sc:"+str(best_f1sc)
    print(text_to_append)
    with open(file_name, "a+") as file_object:
        # Move read cursor to the start of file.
        file_object.seek(0)
        # If file is not empty then append '\n'
        data = file_object.read(100)
        if len(data) > 0:
            file_object.write("\n")
        # Append text at the end of file
        file_object.write(text_to_append)

7904
3723
Acc: 0.4710273279352227
Prec 0.31557200515211026
Recall 0.3397481306572216
F1 0.32721411407805484
7296
4012
Acc: 0.549890350877193
Prec 0.36318479938271603
Recall 0.3485867527244122
F1 0.3557360770626399
Epoch: 1, Train loss: 0.000, Val loss: 0.001, Epoch time = 0.364s
7904
4341
Acc: 0.5492155870445344
Prec 0.39568108091157067
Recall 0.347990850317264
F1 0.37030682487464245
7296
4012
Acc: 0.549890350877193
Prec 0.36318479938271603
Recall 0.3485867527244122
F1 0.3557360770626399
Epoch: 2, Train loss: 0.000, Val loss: 0.001, Epoch time = 0.396s
7904
4394
Acc: 0.555921052631579
Prec 0.40111033809851104
Recall 0.3547985220855658
F1 0.37653575092620967
7296
4012
Acc: 0.549890350877193
Prec 0.36318479938271603
Recall 0.3485867527244122
F1 0.3557360770626399
Epoch: 3, Train loss: 0.000, Val loss: 0.001, Epoch time = 0.373s
7904
4378
Acc: 0.5538967611336032
Prec 0.3984597578347578
Recall 0.3528341756867168
F1 0.37426156109361697
7296
4012
Acc: 0.549890350877193
Prec 0.363184799382716

7904
4814
Acc: 0.6090587044534413
Prec 0.3990469614253141
Recall 0.43073874546673396
F1 0.41428765552119484
7296
4438
Acc: 0.6082785087719298
Prec 0.3962243277175103
Recall 0.4343583485412861
F1 0.41441592645410397
Epoch: 31, Train loss: 0.000, Val loss: 0.001, Epoch time = 0.345s
7904
4396
Acc: 0.5561740890688259
Prec 0.3607066120424398
Recall 0.38551614415966595
F1 0.372698959102329
7296
4086
Acc: 0.5600328947368421
Prec 0.3796665551365641
Recall 0.36067578934232475
F1 0.3699276030392351
Epoch: 32, Train loss: 0.000, Val loss: 0.001, Epoch time = 0.346s
7904
4642
Acc: 0.5872975708502024
Prec 0.3948998117007216
Recall 0.3924951271586919
F1 0.3936937975063419
7296
4451
Acc: 0.6100603070175439
Prec 0.40352564454849543
Recall 0.421836217805937
F1 0.4124778220269704
Epoch: 33, Train loss: 0.000, Val loss: 0.001, Epoch time = 0.356s
7904
4803
Acc: 0.607667004048583
Prec 0.3977811950975265
Recall 0.42604142457399435
F1 0.41142659349574234
7296
4482
Acc: 0.6143092105263158
Prec 0.40380236814

7904
4782
Acc: 0.6050101214574899
Prec 0.5224681393788274
Recall 0.43232227274740737
F1 0.47313967669901685
7296
4348
Acc: 0.5959429824561403
Prec 0.4052208519032526
Recall 0.40024693877978773
F1 0.4027185379231069
Epoch: 61, Train loss: 0.000, Val loss: 0.001, Epoch time = 0.367s
7904
4855
Acc: 0.614245951417004
Prec 0.40373920205782854
Recall 0.4266399551295757
F1 0.41487379243342
7296
4540
Acc: 0.6222587719298246
Prec 0.40884800434208096
Recall 0.4380218004571495
F1 0.4229323987237607
Epoch: 62, Train loss: 0.000, Val loss: 0.001, Epoch time = 0.358s
7904
4926
Acc: 0.6232287449392713
Prec 0.40883702757669
Recall 0.44268069280596123
F1 0.4250863000973112
7296
3484
Acc: 0.47752192982456143
Prec 0.3462002539013978
Recall 0.37788159869159205
F1 0.36134783641713886
Epoch: 63, Train loss: 0.000, Val loss: 0.001, Epoch time = 0.335s
7904
4556
Acc: 0.576417004048583
Prec 0.3762555656832554
Recall 0.40126678380078523
F1 0.3883588962530883
7296
3926
Acc: 0.5381030701754386
Prec 0.355590964808

7904
5042
Acc: 0.6379048582995951
Prec 0.4185019992477195
Recall 0.45251595616663626
F1 0.4348448414180864
7296
4631
Acc: 0.6347313596491229
Prec 0.4152656757896564
Recall 0.4599442436871075
F1 0.4364645622263425
Epoch: 91, Train loss: 0.000, Val loss: 0.001, Epoch time = 0.367s
7904
4980
Acc: 0.6300607287449392
Prec 0.41347114631033094
Recall 0.4457049015567269
F1 0.42898336614548654
7296
4634
Acc: 0.6351425438596491
Prec 0.4192326905236346
Recall 0.44708114525033826
F1 0.43270931079666886
Epoch: 92, Train loss: 0.000, Val loss: 0.001, Epoch time = 0.330s
7904
4966
Acc: 0.6282894736842105
Prec 0.41489429849078174
Recall 0.45208210027465573
F1 0.4326906386858612
7296
4649
Acc: 0.6371984649122807
Prec 0.4177748164303246
Recall 0.45408794445070555
F1 0.43517515863257134
Epoch: 93, Train loss: 0.000, Val loss: 0.001, Epoch time = 0.373s
7904
5033
Acc: 0.6367661943319838
Prec 0.4185083537965237
Recall 0.44659670640343513
F1 0.43209654181111895
7296
4615
Acc: 0.6325383771929824
Prec 0.41463

7904
5154
Acc: 0.6520748987854251
Prec 0.42916834101460904
Recall 0.46632768527900953
F1 0.44697703436766534
7296
4658
Acc: 0.6384320175438597
Prec 0.4188883296685189
Recall 0.45485033326900287
F1 0.43612925565558053
Epoch: 121, Train loss: 0.000, Val loss: 0.001, Epoch time = 0.336s
7904
5074
Acc: 0.6419534412955465
Prec 0.4218067738212305
Recall 0.4569044313675666
F1 0.43865466378881274
7296
4591
Acc: 0.6292489035087719
Prec 0.4121626775175932
Recall 0.44698153779528943
F1 0.42886654914284755
Epoch: 122, Train loss: 0.000, Val loss: 0.001, Epoch time = 0.347s
7904
5091
Acc: 0.6441042510121457
Prec 0.4228930273424609
Recall 0.45677118356073265
F1 0.439179737505667
7296
4521
Acc: 0.6196546052631579
Prec 0.4078854721454799
Recall 0.45686844028558865
F1 0.43098966491027163
Epoch: 123, Train loss: 0.000, Val loss: 0.001, Epoch time = 0.336s
7904
4948
Acc: 0.6260121457489879
Prec 0.4113036577067504
Recall 0.44609816334941604
F1 0.4279949068824261
7296
4415
Acc: 0.6051260964912281
Prec 0.39

...

7904
6161
Acc: 0.7794787449392713
Prec 0.48664259065932264
Recall 0.4805196034016352
F1 0.4835617150833825
7296
4369
Acc: 0.5988212719298246
Prec 0.40529257594139834
Recall 0.42172834459486536
F1 0.41334714245807264
Epoch: 241, Train loss: 0.000, Val loss: 0.001, Epoch time = 0.397s
7904
6123
Acc: 0.774671052631579
Prec 0.4819354356490007
Recall 0.4762598063843127
F1 0.4790808119332168
7296
4365
Acc: 0.5982730263157895
Prec 0.40366864746947345
Recall 0.42175212920146804
F1 0.4125123000871531
Epoch: 242, Train loss: 0.000, Val loss: 0.001, Epoch time = 0.365s
7904
6143
Acc: 0.7772014170040485
Prec 0.4838542562844364
Recall 0.480619539060539
F1 0.48223147326615906
7296
4358
Acc: 0.5973135964912281
Prec 0.40238758975973354
Recall 0.4211018133337608
F1 0.41153205633075346
Epoch: 243, Train loss: 0.000, Val loss: 0.001, Epoch time = 0.355s
7904
6153
Acc: 0.7784665991902834
Prec 0.48470414908274745
Recall 0.48109842643425393
F1 0.48289455696478184
7296
4357
Acc: 0.5971765350877193
Prec 0.404

7904
6147
Acc: 0.7777074898785425
Prec 0.4835608773638976
Recall 0.48050609088537427
F1 0.4820286443568933
7296
4362
Acc: 0.5978618421052632
Prec 0.40680442498269515
Recall 0.4202759444173201
F1 0.4134304723657582
Epoch: 271, Train loss: 0.000, Val loss: 0.001, Epoch time = 0.347s
7904
6171
Acc: 0.7807439271255061
Prec 0.4869513398680608
Recall 0.4861600821155929
F1 0.48655538929750863
7296
4361
Acc: 0.5977247807017544
Prec 0.48642426581041526
Recall 0.4218182301315903
F1 0.4518234366129224
Epoch: 272, Train loss: 0.000, Val loss: 0.001, Epoch time = 0.357s
7904
6134
Acc: 0.7760627530364372
Prec 0.48625976262492515
Recall 0.46940785305673377
F1 0.4776852274916398
7296
4365
Acc: 0.5982730263157895
Prec 0.4060966581611618
Recall 0.42104042058851304
F1 0.41343354600963694
Epoch: 273, Train loss: 0.000, Val loss: 0.001, Epoch time = 0.315s
7904
6149
Acc: 0.7779605263157895
Prec 0.48279435993225356
Recall 0.48273689023267224
F1 0.4827656233721266
7296
4344
Acc: 0.5953947368421053
Prec 0.402

7904
6183
Acc: 0.7822621457489879
Prec 0.4854575211891188
Recall 0.49590830248239165
F1 0.4906272654972516
7296
4324
Acc: 0.5926535087719298
Prec 0.4017380677574853
Recall 0.41689251878577344
F1 0.4091750240282466
Epoch: 301, Train loss: 0.000, Val loss: 0.001, Epoch time = 0.334s
7904
6170
Acc: 0.7806174089068826
Prec 0.8292492053683729
Recall 0.47345815749767967
F1 0.6027674550274179
7296
4341
Acc: 0.594983552631579
Prec 0.40143346543232
Recall 0.41911578416192635
F1 0.41008410338977924
Epoch: 302, Train loss: 0.000, Val loss: 0.002, Epoch time = 0.346s
7904
6185
Acc: 0.7825151821862348
Prec 0.652592329598935
Recall 0.4972141584625593
F1 0.5644047921971749
7296
4343
Acc: 0.5952576754385965
Prec 0.5138301788116669
Recall 0.41955854729917413
F1 0.46193367746997455
Epoch: 303, Train loss: 0.000, Val loss: 0.001, Epoch time = 0.343s
7904
6136
Acc: 0.7763157894736842
Prec 0.5821826715024231
Recall 0.4723695613398924
F1 0.521558562189128
7296
4371
Acc: 0.5990953947368421
Prec 0.40756305499

7904
6176
Acc: 0.7813765182186235
Prec 0.7529975524609599
Recall 0.49272966277917357
F1 0.595674920734731
7296
4327
Acc: 0.5930646929824561
Prec 0.4439545440709718
Recall 0.4178800626742525
F1 0.43052286656606786
Epoch: 331, Train loss: 0.000, Val loss: 0.001, Epoch time = 0.366s
7904
6163
Acc: 0.7797317813765182
Prec 0.6363087496036964
Recall 0.48049891936649197
F1 0.5475350412841431
7296
4357
Acc: 0.5971765350877193
Prec 0.40674117535234205
Recall 0.41998088751489954
F1 0.4132550163736504
Epoch: 332, Train loss: 0.000, Val loss: 0.001, Epoch time = 0.337s
7904
6177
Acc: 0.781503036437247
Prec 0.8201422996748553
Recall 0.48711787213240304
F1 0.6112111123389967
7296
4348
Acc: 0.5959429824561403
Prec 0.4037452500379997
Recall 0.4195288638253151
F1 0.4114857571033304
Epoch: 333, Train loss: 0.000, Val loss: 0.001, Epoch time = 0.326s
7904
6168
Acc: 0.7803643724696356
Prec 0.6430294505858569
Recall 0.48947787566353346
F1 0.555843979401407
7296
4334
Acc: 0.5940241228070176
Prec 0.401982230

7904
6201
Acc: 0.7845394736842105
Prec 0.6513972456909686
Recall 0.49359746679796995
F1 0.5616236072449865
7296
4365
Acc: 0.5982730263157895
Prec 0.4737568274894323
Recall 0.4213199731163511
F1 0.4460024294819059
Epoch: 361, Train loss: 0.000, Val loss: 0.001, Epoch time = 0.343s
7904
6198
Acc: 0.78415991902834
Prec 0.5872539534442439
Recall 0.48970086548679076
F1 0.5340591159574013
7296
4345
Acc: 0.5955317982456141
Prec 0.443976150906844
Recall 0.41899875233351863
F1 0.431125986624416
Epoch: 362, Train loss: 0.000, Val loss: 0.002, Epoch time = 0.347s
7904
6212
Acc: 0.7859311740890689
Prec 0.6800782575459537
Recall 0.49654979060859983
F1 0.5740007930484889
7296
4350
Acc: 0.5962171052631579
Prec 0.5273905184441651
Recall 0.4214768053105904
F1 0.468522553786232
Epoch: 363, Train loss: 0.000, Val loss: 0.002, Epoch time = 0.367s
7904
6171
Acc: 0.7807439271255061
Prec 0.5840889242756915
Recall 0.48515841321128933
F1 0.5300469699403554
7296
4327
Acc: 0.5930646929824561
Prec 0.3987759857869